  **Assignment 2**
  
  Name: Tao He
  
  Discussion Partner: Yifan Zhang

**Assignment 2**: (One week, due in Gradescope at midnight 9/28 with same grace period and late policy as in PS 1)

In this assignment, we are going to try building language models with 
the data we collected from various sources.

In the first half, we are going to analyze our twitter data with NLTK. 
There are several tasks we would like to you to finish during this process:

1. Preprocess the raw twitter data and make them into a format that
language models in NLTK can train with.
2. Train uni-gram, bi-gram and tri-gram models with Add-one smoothing.
3. Compute perplexity to evaluate our language models based on different test sets.
4. Generate new sentences with our language models based on the trained data.
5. Perform sentiment analysis on our scraped data.

In the following sections, we are going to provide a code template to allow you
to complete them step by step.

Here's a [general guide](https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook) of how to build language model with NLTK, please refer to this guide from time to time to see what you missed.

Please submit this code with your implementaton and outputs. **Please indicate which students, if any, you consulted with as you completed this assignment.** 

First, please go back to the code of our first lab section. Scrape 10000 tweets
which: football lang:en -has:mentions -has:links -is:retweet
1. mentions 'fishing'
2. is written in English
3. does not mention any other twitter account (i.e. @).
4. does not contain links.
5. is not a re-tweet.

Then, scrape 10000 tweets with the same rules above but mention 'football' instead of 'fishing' this time.

Save the scraped tweets in separate files, one for 'fishing' tweets
and one for 'football'.

## Task 1

First, let's try loading our scraped data. To begin with, let's load our 'fishing' data. You may change the following
function as necessary.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install tweepy
!pip3 install tweepy --upgrade

import pandas as pd
import numpy as np
import re
import tweepy
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAAl1hAEAAAAAra3%2BdGSwRPN3G2BB45Ea8fDSny4%3DXWlNMYMgowFpQWpqJGYjtjpZmUbYQxyrY3IZeRAddDCZm0S6Tm') # replace with your bearer token here.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0


In [ ]:
# Language is English.
# no mentions, no links, not re-tweet.
query = 'fishing lang:en -has:mentions -has:links -is:retweet'
tweets = list(tweepy.Paginator(client.search_recent_tweets, query=query, tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=10000))
print("{} tweets are collected.".format(len(tweets)))

7957 tweets are collected.


In [ ]:
import csv

driveFolderDirectory = '/content/drive/My Drive/CS505/tweetsFishing.csv' 
pathToSave = driveFolderDirectory

with open(pathToSave, 'w', newline='') as csvfile:
  fieldnames = ['idx','tweetId', 'tweetText']
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
  writer.writeheader()
  for i,tweet in enumerate(tweets):
    writer.writerow({'idx': i, 'tweetId': tweet.id,'tweetText': tweet.data['text']})

In [ ]:
def loadTextFromCSV(csvPath):
  tweetDict = {}
  with open(csvPath, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      tweetDict[int(row['idx'])] = row['tweetText']
  return tweetDict

# load fishing tweet data here:
csvPathFish = "/content/drive/My Drive/CS505/tweetsFishing.csv"
rawTweetDictFish = loadTextFromCSV(csvPathFish)

# print tweet dictionary
# print("rawTweetDictFish: ",rawTweetDictFish)

## Task 2

Next, we are going to pre-process texts with NLTK library.

Install NLTK library if it's not in your Google Colab space.

Download 'punkt' specifically for sentence segmentation.

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

We preprocess our tweet data with the following steps:
1. Split data into training and testing splits. (**80%** tweets for training and **20%** tweets for testing) 
2. Sentence segmentation/spliting.
3. Lower-case all words in the sentences.
4. Tokenization (you should use TweetTokenizer from NLTK.tokenize)
5. Padding with begin-of-sentence and end-of-sentence symbols 

You may refer to the following materials:
1. [Sentence segmentation](https://www.nltk.org/api/nltk.tokenize.html). 
2. [String Lower case](https://www.w3schools.com/python/ref_string_lower.asp).
3. [Tweet Tokenization](https://www.nltk.org/api/nltk.tokenize.casual.html).
4. [Padding tokenized sentences](https://www.nltk.org/_modules/nltk/lm/preprocessing.html). Particularly, please look at function 'padded_everygram_pipeline'.

### 2.1 Split Data

In [ ]:
def preprocess(rawTweetDataDict):
    #Input: a dictionary contains raw tweet data scraped from Tweeter
    #Output: two lists of tweet sentences (train/test), but each tweet sentence is
    #     represented in the form of tokens.

    rawTweetDataFrame = pd.DataFrame.from_dict(rawTweetDictFish, orient ='index') # first, change dic to dataframe

    training_data     = rawTweetDataFrame.sample(frac=0.8, random_state=25) # set seed
    testing_data      = rawTweetDataFrame.drop(training_data.index)

    # convert to list output
    train_list        = training_data.values.tolist()
    test_list         = testing_data.values.tolist()

    train = []
    test  = [] 
    for i in train_list: 
      train.append(i[0])
    for i in test_list: 
      test.append(i[0])

    return(train, test)

In [ ]:
train, test = preprocess(rawTweetDictFish)

### 2.2 Sentence Segmentation

In [ ]:
from nltk import sent_tokenize, word_tokenize
def sentenceSegmentation(tweets):
    #Input: a string of raw tweet
    #Output: a list of strings, each element in the list is a segmented sentence

  tweet_segment = []
  for str in tweets: 
    txts = sent_tokenize(str)
    for sent in txts: 
      tweet_segment.append(sent)
  
  return(tweet_segment)

In [ ]:
train = sentenceSegmentation(train)
test  = sentenceSegmentation(test)

### 2.3 String Lower Case

In [ ]:
def sentenceLowerCase(sentence):
  #Input: a string of sentence
  #Output: a string of sentence, but all words in the sentence are lower-cased.
  return(sentence.lower())

In [ ]:
# define lower list
train_lower = []
test_lower  = []

for sent in train:
  lower = sentenceLowerCase(sent)
  # use code from the last assignment
  lower = re.sub(r'[^a-zA-Z,.?!():;\'"]', ' ', lower)
  lower = re.sub(r'\n', '', lower)
  train_lower.append(lower)

for sent in test:
  lower = sentenceLowerCase(sent)
  lower = re.sub(r'[^a-zA-Z,.?!():;\'"]', ' ', lower)
  lower = re.sub(r'\n', '', lower)
  test_lower.append(lower)

### 2.4 Tokenization

In [ ]:
# https://www.nltk.org/api/nltk.tokenize.casual.html

from nltk.tokenize import TweetTokenizer
def sentenceTokenization(sentence):
    #Input: a string of sentence
    #Output: a list of tokens that belong to the sentence.
    tknzr = TweetTokenizer()
    return (tknzr.tokenize(sentence))

In [ ]:
train_token = []
test_token  = []

for sent in train_lower:
  token = sentenceTokenization(sent)
  train_token.append(token)

for sent in test_lower:
  token = sentenceTokenization(sent)
  test_token.append(token)

### 2.5 Padding with begin-of-sentence and end-of-sentence symbols

In [ ]:
# https://www.nltk.org/_modules/nltk/lm/preprocessing.html
from nltk.util import pad_sequence

# list(pad_sequence(text[0],
#                   pad_left=True, left_pad_symbol="<s>",
#                   pad_right=True, right_pad_symbol="</s>",
#                   n=2)) # The n order of n-grams, if it's 2-grams, you pad once, 3-grams pad twice, etc. 

train_data = []
test_data  = []

for str in train_token:
  token = list(pad_sequence(str,
                            pad_left = True, left_pad_symbol = "<s>",
                            pad_right = True, right_pad_symbol = "</s>",
                            n = 2))
  train_data.append(token)

for str in test_token:
  token = list(pad_sequence(str,
                            pad_left = True, left_pad_symbol = "<s>",
                            pad_right = True, right_pad_symbol = "</s>",
                            n = 2))
  test_data.append(token)

## Task 3

Next, we build our n-gram model with our pre-processed data.
First we need to pad our data with padded_everygram_pipeline. Then, we train our n-gram model with add-one smoothing using the corresponding functions in NLTK.

Related materials:
1. [Padding](https://www.nltk.org/_modules/nltk/lm/preprocessing.html)
2. [N-gram Language Model](https://www.nltk.org/api/nltk.lm.html)

Let's train unigram, bigram and trigram models with our train data split.

In [ ]:
# https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline

# Here's a template you may want to start with
def trainNGramAddOneSmoothing(trainData,ngram):
  # Input: a list of tweet sentences, each element is a list of tokens; n for ngram model
  # Output: a n-gram model with add-one smoothing trained on your input data. 
  train, vocab = padded_everygram_pipeline(ngram, trainData)
  return(train, vocab)

unigram_train_data, unigram_train_vocab = trainNGramAddOneSmoothing(train_token, 1)
bigram_train_data, bigram_train_vocab   = trainNGramAddOneSmoothing(train_token, 2)
trigram_train_data, trigram_train_vocab = trainNGramAddOneSmoothing(train_token, 3)

## Task 4

Now we apply our analysis on the trained model. 

First, compute the average perplexity of your tri-gram model on the sentences of our test data.

[How to compute perplexity](https://www.nltk.org/api/nltk.lm.html)

Next, load the tweet data of 'football' instead, and compute the perplexity of your 'fishing' model on the football tweets. 

**Why is there a difference between the two perplexities, what causes it?**
(Please answer in a text cell.)

Average perplexity of the trigram model on the sentences of fishing data =  4.710520750462658.

Average perplexity of the trigram model on the sentences of football data = 7.028510179325636.

Although both of them are tri-gram model, we used different train data sets so that the models we got are definitely different. 

### 4.1 tweet data of "fishing"

In [ ]:
def computePerplexity(model,testData):
    # Input: your model; the testing data
    # Output: average perplexity of the model on your testing data.
    test = list(trigrams(pad_both_ends(testData, n=3)))
    perplexity = model.perplexity(test)
    return(perplexity)

In [ ]:
from nltk.lm import MLE
from nltk.util import bigrams, trigrams

trimodel_fishing = MLE(3)
len(trimodel_fishing.vocab)

0

In [ ]:
trimodel_fishing.fit(trigram_train_data, trigram_train_vocab)
print(trimodel_fishing.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 14461 items>


In [ ]:
len(trimodel_fishing.vocab)

14461

In [ ]:
score = []
for sent in test_token:
    score.append(computePerplexity(trimodel_fishing, sent))
    
# remove 'inf' term in score
filtered_score = []

for i in score:
    if not ((np.isnan(i)) or (i == float('Inf')) or (i == float('-Inf'))):
        filtered_score.append(i)
        
print('Average perplexity of the trigram model on the sentences of fishing data = ', sum(filtered_score)/len(filtered_score))
# 1. the perplexity of your model on your testing data of 'fishing' tweets.

Average perplexity of the trigram model on the sentences of fishing data =  4.710520750462658


### 4.2 tweet data of "football"

In [ ]:
# Language is English.
# no mentions, no links, not re-tweet.
Query = 'football lang:en -has:mentions -has:links -is:retweet'
football = list(tweepy.Paginator(client.search_recent_tweets, query=Query, tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=10000))
print("{} tweets are collected.".format(len(football)))

10000 tweets are collected.


In [ ]:
driveFolderDirectory = '/content/drive/My Drive/CS505/tweetsFootball.csv' 
pathToSave = driveFolderDirectory

with open(pathToSave, 'w', newline='') as csvfile:
  fieldnames = ['idx','tweetId', 'tweetText']
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
  writer.writeheader()
  for i,tweet in enumerate(football):
    writer.writerow({'idx': i, 'tweetId': tweet.id,'tweetText': tweet.data['text']})

In [ ]:
def loadTextFromCSV(csvPath):
  tweetDict = {}
  with open(csvPath, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      tweetDict[int(row['idx'])] = row['tweetText']
  return tweetDict

csvPathFootball = "/content/drive/My Drive/CS505/tweetsFootball.csv"
rawTweetDictFootball = loadTextFromCSV(csvPathFootball)

# print("rawTweetDictFootball: ",rawTweetDictFootball)

In [ ]:
def preprocessFootball(rawTweetDataDict):
    #Input: a dictionary contains raw tweet data scraped from Tweeter
    #Output: two lists of tweet sentences (train/test), but each tweet sentence is
    #     represented in the form of tokens.

    rawTweetDataFrame = pd.DataFrame.from_dict(rawTweetDictFootball, orient ='index') # first, change dic to dataframe

    training_data     = rawTweetDataFrame.sample(frac=0.8, random_state=25) # set seed
    testing_data      = rawTweetDataFrame.drop(training_data.index)

    # convert to list output
    train_list        = training_data.values.tolist()
    test_list         = testing_data.values.tolist()

    train = []
    test  = [] 
    for i in train_list: 
      train.append(i[0])
    for i in test_list: 
      test.append(i[0])

    return(train, test)

In [ ]:
# Split Data
train_football, test_football = preprocessFootball(rawTweetDictFootball)
# Sentence Segmentation
train_football = sentenceSegmentation(train_football)
test_football  = sentenceSegmentation(test_football)
# String Lower Case
football_train_lower = []
football_test_lower  = []

for sent in train_football:
  lower = sentenceLowerCase(sent)
  # use code from the last assignment
  lower = re.sub(r'[^a-zA-Z,.?!():;\'"]', ' ', lower)
  lower = re.sub(r'\n', '', lower)
  football_train_lower.append(lower)

for sent in test_football:
  lower = sentenceLowerCase(sent)
  lower = re.sub(r'[^a-zA-Z,.?!():;\'"]', ' ', lower)
  lower = re.sub(r'\n', '', lower)
  football_test_lower.append(lower)
# Tokenization
football_train_token = []
football_test_token  = []

for sent in football_train_lower:
  token = sentenceTokenization(sent)
  football_train_token.append(token)

for sent in football_test_lower:
  token = sentenceTokenization(sent)
  football_test_token.append(token)

In [ ]:
# train unigram, bigram and trigram models with our train data split.
f_unigram_train_data, f_unigram_train_vocab = trainNGramAddOneSmoothing(football_train_token, 1)
f_bigram_train_data,  f_bigram_train_vocab   = trainNGramAddOneSmoothing(football_train_token, 2)
f_trigram_train_data, f_trigram_train_vocab = trainNGramAddOneSmoothing(football_train_token, 3)

In [ ]:
trimodel_football = MLE(3)
trimodel_football.fit(f_trigram_train_data, f_trigram_train_vocab)

football_score = []
for sent in football_test_token:
    football_score.append(computePerplexity(trimodel_football, sent))
    
# remove 'inf' term in score
football_filtered_score = []

for i in football_score:
    if not ((np.isnan(i)) or (i == float('Inf')) or (i == float('-Inf'))):
        football_filtered_score.append(i)
        
print('Average perplexity of the trigram model on the sentences of football data = ', sum(football_filtered_score)/len(football_filtered_score))
# 2. the perplexity of your model on your data of 'football' tweets.

Average perplexity of the trigram model on the sentences of football data =  7.028510179325636


## Task 5

Next, generate 10 tweets using each of your language models (unigram, bigram, trigram). The generated tweets needs to be in string format instead of tokens, also the string should be without padding.

[Generate new sentences with your model.](https://www.nltk.org/api/nltk.lm.html)

[Detokenize your generated sentences](https://www.nltk.org/howto/tokenize.html)

In [ ]:
unimodel_fishing = MLE(1)
unimodel_fishing.fit(unigram_train_data, unigram_train_vocab)

bimodel_fishing = MLE(2)
bimodel_fishing.fit(bigram_train_data, bigram_train_vocab)

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
def generateNewSentence(model, sent_length, randomSeed):
  # Input: your model; random seed that get you different generated sentence
  # Output: a new sentence generated by your model, but in a string format instead of tokens.
  sent = model.generate(sent_length, random_seed = randomSeed) 
  de = TreebankWordDetokenizer().detokenize(sent)  # detokenize
  sent_clr = re.sub("<s>|</s>", "", de)
  return(sent_clr)

In [ ]:
# Make loops to generate 10 tweets for each of your model (unigram, bigram and trigram)
unisentence = []
bisentence  = []
trisentence = []
for i in range(10):
  unisentence.append(generateNewSentence(unimodel_fishing, 10, i))
  bisentence.append(generateNewSentence(bimodel_fishing, 10, i))
  trisentence.append(generateNewSentence(trimodel_fishing, 10, i))

In [ ]:
unisentence

['the shadow going close in games species ease i looking',
 "add the shouts chest i'm he's occasion started:,",
 'when weeks . . the roc omfg engaging mechanism men',
 "by it's fishing me my . (the clothing but",
 'but? friends and . fun up swimming sidearm boat',
 'my s study wdym ryohgo use, household we now',
 'straight the i combos! of husbands she fishing small',
 'feel amp nymphing . it fishing . in, had',
 'boy who a placement . carrying yourselves became no his',
 'hook fishing agriculture thirst! im to . just more']

In [ ]:
bisentence

['the same line after it is taking .  into',
 ' the same is kind of soft torture . ',
 'went to assist because they said so future as others',
 'a leopard chat on sunday!!  amp;',
 'a cabin above all about his tiny tempor pet coyote',
 'lest china often used to the announcers are while it',
 'shots of military and " liquorice .  dead of',
 'books, not a lake? " grandad, hunting',
 'a while...it .  yours for our friend',
 'for fishing . !  to be in something']

In [ ]:
trisentence

['suppose the same is true of everything in life.',
 '         ',
 'we were bros...that looks very much related to',
 ' and i m not!!   ',
 "  i didn't but i threw mine on anyway",
 'in the sea was calm, might have to learn',
 'resume writing, fishing!     ',
 'a cool front rolled in, after a few hours',
 " we're about to become a second...maybe also",
 'edna collection .       ']

## Task 6

Lastly, we want to perform sentiment analysis on our collected data.
This time we will use VADER.

Please check out the following material:

[Sentiment analysis with VADER](https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/)

Then do the following:

1. Compute the ratios of positive and negative sentences in your collected data.
2. Compute the average compound sentiment of the tweets for 'fishing' and 'football'. Are they generally positive or negative? (Answer in a text cell.)
3. Compute the top 10 non stop words from positive tweets of 'fishing'. Please check out [here](https://www.geeksforgeeks.org/removing-stop-words-nltk-python/) to find out how to remove stop words in your sentences. The top 10 words shall also not include puncutations, including symbols like parenthesis ' ", or ...
You can refer to [here](https://docs.python.org/3/library/string.html) to see how to exclude them (still there will be special cases, please remember to remove them as well.)

### 6.1 compute the ratios of positive and negative sentences.

In [ ]:
# install VADER
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 27.3 MB/s 


In [ ]:
fishing_train_clr = []
fishing_test_clr  = []

for sent_list in train_token: 
    word_list = []
    for word in sent_list: 
        word_clr = re.sub(r'[^A-Za-z]', '', word)
        word_list.append(word_clr)
    word_list = list(filter(None, word_list))
    fishing_train_clr.append(word_list)
        
for sent_list in test_token: 
    word_list = []
    for word in sent_list: 
        word_clr = re.sub(r'[^A-Za-z]', '', word)
        word_list.append(word_clr)
    word_list = list(filter(None, word_list))
    fishing_test_clr.append(word_list)  


football_train_clr = []
football_test_clr  = []

for sent_list in football_train_token: 
    word_list = []
    for word in sent_list: 
        word_clr = re.sub(r'[^A-Za-z]', '', word)
        word_list.append(word_clr)
    word_list = list(filter(None, word_list))
    football_train_clr.append(word_list)
        
for sent_list in football_train_token: 
    word_list = []
    for word in sent_list: 
        word_clr = re.sub(r'[^A-Za-z]', '', word)
        word_list.append(word_clr)
    word_list = list(filter(None, word_list))
    football_test_clr.append(word_list)


fishing_clr = fishing_train_clr + fishing_test_clr
football_clr = football_train_clr + football_test_clr

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
import nltk
# need to download 'stopwords' before using it.
nltk.download('stopwords')
from nltk.corpus import stopwords

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words = set(stopwords.words('english'))

def removeStopWords(sentence_list): 
    filtered_token_list = []
    for sent in sentence_list:
        filtered_token = []
        for w in sent:
            if w not in stop_words:
                filtered_token.append(w)
        filtered_token_list.append(filtered_token)
    return(filtered_token_list)


fishing_filtered_token  = removeStopWords(fishing_clr) # remove stop words
fishing_filtered_token = [ele for ele in fishing_filtered_token if ele != []] # remove empty list
football_filtered_token = removeStopWords(football_clr)
football_filtered_token = [ele for ele in football_filtered_token if ele != []]

In [ ]:
# https://github.com/syp1997/CS505-NLP-2021-Spring/blob/main/homework2/homework2.ipynb

def computeSentimentOfSentences(sentenceData):
    senti_analyzer = SentimentIntensityAnalyzer()
    avg_compound_score = []
    i = 0
    for sentence in sentenceData:
    #     print(i)
        compound_score = []
        num_postive = 0
        num_neutral = 0
        num_negative = 0
        pos_twitter = []
        neu_twitter = []
        neg_twitter = []
        for text in sentence:
            sentiment = senti_analyzer.polarity_scores(text)
            score = sentiment['compound']
            compound_score.append(score)
            if score >= 0.05:
                num_postive += 1
                pos_twitter.append(text)
            elif score < 0.05 and score > -0.05:
                num_neutral += 1
                neu_twitter.append(text)
            elif score <= -0.05:
                num_negative += 1
                neg_twitter.append(text)
            else:
                print("no match!!")
        avg_score = (np.array(compound_score)).mean()
        avg_compound_score.append(avg_score)
        i = i+1
    
    sentenceArray = np.array(sentenceData, dtype=object)
    avg_compound_scoreArray = np.array(avg_compound_score)
    positive_index = np.where(avg_compound_scoreArray > 0)[0]
    sentencePositive = list(sentenceArray[positive_index])
    
    return(avg_compound_score, sentencePositive)

In [ ]:
fishing_score, fishing_positive   = computeSentimentOfSentences(fishing_filtered_token)
football_score, football_positive = computeSentimentOfSentences(football_filtered_token)
fishing_score_array  = np.array(fishing_score)
football_score_array = np.array(football_score)

print("fishing_positive_ratio: {} ;  fishing_negative_ratio: {}".format(round(len(np.where(fishing_score_array > 0)[0])/len(fishing_score_array), 3), round(len(np.where(fishing_score_array < 0)[0])/len(fishing_score_array), 3)))
print("football_positive_ratio: {} ; football_negative_ratio: {}".format(round(len(np.where(football_score_array > 0)[0])/len(football_score_array), 3), round(len(np.where(football_score_array < 0)[0])/len(football_score_array), 3)))

fishing_positive_ratio: 0.353 ;  fishing_negative_ratio: 0.199
football_positive_ratio: 0.407 ; football_negative_ratio: 0.19


### 6.2 Compute the average compound sentiment of the tweets for 'fishing' and 'football'. Are they generally positive or negative? (Answer in a text cell.)

They generally positive.

In [ ]:
print("average sentiment compound of fishing: ", sum(fishing_score)/len(fishing_score))
print("average sentiment compound of football: ", sum(football_score)/len(football_score))

average sentiment compound of fishing:  0.017127030263975405
average sentiment compound of football:  0.019431180960325943


### 6.3 Compute the top 10 non stop words from positive tweets of 'fishing'.

In [ ]:
uniques = []
for sentence in fishing_positive:
    for word in sentence:
        if word not in uniques:
            uniques.append(word)
            
counts = []
for unique in uniques:
    count = 0
    for sentence in fishing_positive: 
        for word in sentence:
            if word == unique:
                count += 1
    counts.append((count, unique))

In [ ]:
counts.sort()
counts.reverse()
print('top 10 words in positive tweets: ')
for i in range(min(10, len(counts))):
    count, word = counts[i]
    print('%s: %d;' % (word, count))

top 10 words in positive tweets: 
fishing: 3149;
like: 834;
go: 398;
good: 334;
want: 325;
love: 303;
fish: 265;
im: 231;
one: 217;
would: 194;
